In [1]:
import numpy as np
import pandas as pd

In [2]:
J=pd.read_csv("hw3_JNJ.csv")
J=J.set_index("Date")
J=J["Adj Close"]
M=pd.read_csv("hw3_MRK.csv")
M=M.set_index("Date")
M=M["Adj Close"]
P=pd.read_csv("hw3_PFE.csv")
P=P.set_index("Date")
P=P["Adj Close"]

In [3]:
P.tail(5)

Date
2021-02-22    31.986647
2021-02-23    31.659872
2021-02-24    31.510492
2021-02-25    31.575851
2021-02-26    31.267750
Name: Adj Close, dtype: float64

In [4]:
r1 = np.log(J) - np.log(J.shift(1))
r2 = np.log(M) - np.log(M.shift(1))
r3 = np.log(P) - np.log(P.shift(1))

In [5]:
r1_ewm = r1.ewm(alpha=0.06, adjust=True)
r2_ewm = r2.ewm(alpha=0.06, adjust=True)
r3_ewm = r3.ewm(alpha=0.06, adjust=True)

In [6]:
ewm_mean1 = r1_ewm.mean()
ewm_std1 = r1_ewm.std()
ewm_mean2 = r2_ewm.mean()
ewm_std2 = r2_ewm.std()
ewm_mean3 = r3_ewm.mean()
ewm_std3 = r3_ewm.std()

In [7]:
w1=1.2*10**6*J
w2=2.5*10**6*M
w3=3.5*10**6*P

In [8]:
variance_p=w1*w1*ewm_std1**2+w2*w2*ewm_std2**2+w3*w3*ewm_std3**2+2*w1*w2*r1_ewm.cov(other=r2)+2*w1*w3*r1_ewm.cov(other=r3)+2*w3*w2*r2_ewm.cov(other=r3)

In [9]:
#Q1 a
#1 day 99% VaR
2.33*np.sqrt(variance_p[-1])

9864507.366944488

In [10]:
J1=J[-1]*np.exp(r1)
M1=M[-1]*np.exp(r2)
P1=P[-1]*np.exp(r3)

In [11]:
portfolio_historical=1.2*10**6*(J1-J[-1])+2.5*10**6*(M1-M[-1])+3.5*10**6*(P1-P[-1])
portfolio_historical=portfolio_historical.dropna()
portfolio_historical

Date
2019-03-04   -1.113799e+06
2019-03-05    5.730132e+05
2019-03-06   -4.062330e+06
2019-03-07   -2.949543e+06
2019-03-08   -2.856621e+06
                  ...     
2021-02-22    6.584782e+05
2021-02-23   -3.550233e+06
2021-02-24    1.967940e+06
2021-02-25    5.236620e+05
2021-02-26   -1.014895e+07
Name: Adj Close, Length: 502, dtype: float64

In [12]:
#Q1 b
#historical 99% VaR
np.percentile(np.array(portfolio_historical),0.99)
#Therefore the VaR is 19146342

-19403874.928801242

#Q1 (c)
I would recommend the parametric approach to the risk manager because for the historical approach we actually do not have much data, so if there are only few very exceptional cases happened in the past, then when we get our 99% VaR prediction based on it, we may get very unreasonable price changes. However, for the parametric approach when we do the prediction, we make more use of the whole datasets' properties instead of focusing on one or two very exceptional cases happend in the past. Besides, it is also logical to assume that the recent price changes could play a more important role when predicting tomorrow's price change.